In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.keras as tf_keras

In [7]:
# base_dir = "data-files/aclImdb"
base_dir = "D:\\instructor-och\\data-files\\aclImdb"
train_dataset = tf_keras.utils.text_dataset_from_directory(base_dir + "\\train", batch_size=32)
test_dataset = tf_keras.utils.text_dataset_from_directory(base_dir + "\\test", batch_size=32)
review_only_dataset = train_dataset.map(lambda review, label: review)

Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [5]:
for X, y in train_dataset:
    print(X.shape, y.shape)
    print(X[0], y[0])
    break

(32,) (32,)
tf.Tensor(b"... mainly because Ju-on 2 boasts an outrageous FORTY minutes' worth of material literally taken straight out of the first Ju-on - and when you consider that the sequel only runs for 76 minutes, that leaves you with 36 original minutes' worth of film. Ho-hum. I found that deeply irritating - as if viewers simply wouldn't remember the same stuff! - not to mention dull, having to watch it all over again.<br /><br />OK, that complaint aside, the byline for Ju-on 2 was that it was supposed to explain a lot of the unanswered questions from the first movie, which frankly, over 36 minutes, simply doesn't go far enough to making any kind of sense of the original's highly convoluted storyline.<br /><br />There are, however, some really nice new horror sequences which show how good the film might have been, had it had some time to develop; and some of the questions raised by the original - some, but not all - are answered.<br /><br />So in conclusion - if you loved the fi

In [84]:
# 텍스트 -> 숫자 인코딩 (BOW, 단어번호벡터, ...)
text_vectorizer = tf_keras.layers.TextVectorization(max_tokens=20000,  # 사용할 단어 갯수
                                                    output_mode="int", # 출력은 단어 사전의 번호
                                                    output_sequence_length=300) # 각 문장의 길이
text_vectorizer.adapt(review_only_dataset)

In [85]:
for X, y in train_dataset:    
    d = text_vectorizer(X)
    print(d.shape)    
    print( text_vectorizer(X) )
    break

(32, 300)
tf.Tensor(
[[  196   744  2069 ...     0     0     0]
 [  241   173 15476 ...     0     0     0]
 [   10   418    78 ...     0     0     0]
 ...
 [   51    10    86 ...   460    37     7]
 [   15    10  1751 ...     0     0     0]
 [   10   321   761 ...     0     0     0]], shape=(32, 300), dtype=int64)


In [ ]:
text_vectorizer.get_vocabulary()

In [89]:
# 단어 값 -> 압축된 단어 벡터 ( 과정 학습 ), 한 행의 문장 -> 여러 행의 단어 벡터
# input_dim : 총 단어 갯수, output_dim : 한 단어를 표현하는 vector
input = tf_keras.layers.Input(shape=(None,))
output = tf_keras.layers.Embedding(input_dim=20000, output_dim=100)(input)

embedding_model = tf_keras.models.Model(input, output)

In [88]:
input_dataset = train_dataset.map(lambda review, label: (text_vectorizer(review), label) )

# for X, y in input_dataset:
#     print(X, y)

all_data = []
for x, y in input_dataset.as_numpy_iterator(): # tensorflow tensor -> numpy ndarray
    all_data.append(x)

review_only_input_dataset = np.concatenate(all_data)

In [90]:
print(review_only_input_dataset.shape)
review_only_input_dataset[:5]

(25000, 300)


array([[11492,   601,   917, ...,    78,     2,  3866],
       [   51,    10,    14, ...,    30,   533,  1825],
       [  696,     3,  4723, ...,     0,     0,     0],
       [   10,    14, 10124, ...,     0,     0,     0],
       [  330, 10987,   149, ...,     0,     0,     0]], dtype=int64)

In [91]:
# 배치크기 * 단어 갯수 -> 배치크기 * 단어 갯수 * 단어표현크기
embeded_dataset = embedding_model(review_only_input_dataset) 

In [92]:
embeded_dataset.shape

TensorShape([25000, 300, 100])